# Урок 1. База данных NCBI

## Домашнее задание.

Скачайте файл `curated_gene_disease_associations.tsv.gz` на ПК. В этом файле содержится список генов и ассоциированных с ними болезней. Что требуется сделать:
- Выберите любое из заболеваний, которое есть в списке.
- В отдельную таблицу извлеките все гены, которые связаны с избранным заболеванием.
- Для одного из генов извлеките отдельно информацию о 2–3 характеристиках на выбор, а также добавьте вырезанную по координатам последовательность гена.

Пример выполнения — в файле Task1.ipynb. Сделанное практическое задание предоставляется в формате .ipynb.

Файл ipynb. для скачивания https://colab.research.google.com/drive/1gpyi5Kdw8xEJccm1Xzsjq95JF_V7fLAC

## Задание выполнено командой M4 Team.
__Состав команды:__  
 - Мгер Парунакян
 - Михаил Демин
 - Марат Закиров
 - Марина Дружинина

In [ ]:
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython
except ImportError:
    pass


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.7 MB/s eta 0:00:00


In [ ]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "A.N.Other@example.com"
import pandas as pd
import xml.etree.ElementTree as ET

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Colab_Notebooks/GB'

/content/drive/MyDrive/Colab_Notebooks/GB


In [ ]:
dis = pd.read_table('gene_disease_associations.tsv')
dis

,gene_pli,score,diseaseid,gene_dsi,protein_class_name,geneid,disease_class,disease_type,uniprotid,disease_class_name,disease_name,el,year_initial,source,ei,gene_dpi,protein_class,year_final,gene_symbol,disease_semantic_type
0,1.643200e-05,1.00,C0001080,0.391,Kinase,2261,C16;C05,disease,P22607,"Congenital, Hereditary, and Neonatal Diseas...",Achondroplasia,definitive,1994.0,CURATED,0.974,0.846,DTO_03300101,2020.0,FGFR3,Congenital Abnormality
1,1.040000e-05,0.80,C0000744,0.505,Transporter,4547,C16;C18,disease,P55157,"Congenital, Hereditary, and Neonatal Diseas...",Abetalipoproteinemia,NaN,1985.0,CURATED,0.982,0.846,DTO_05007405,2019.0,MTTP,Disease or Syndrome
2,9.973100e-01,0.80,C0001193,0.380,Kinase,2263,C16;C05,disease,P21802,"Congenital, Hereditary, and Neonatal Diseas...",Apert syndrome,NaN,1994.0,CURATED,0.976,0.808,DTO_03300101,2019.0,FGFR2,Congenital Abnormality
3,5.323500e-01,0.70,C0001418,0.236,Transcription factor,7157,C04,group,P04637,Neoplasms,Adenocarcinoma,NaN,1986.0,CURATED,0.975,0.962,DTO_05007542,2020.0,TP53,Neoplastic Process
4,8.460200e-01,0.69,C0001126,0.522,Transporter,6521,C16;C18;C13;C12,phenotype,P02730,"Congenital, Hereditary, and Neonatal Diseas...",Renal tubular acidosis,NaN,1996.0,CURATED,1.000,0.731,DTO_05007405,2013.0,SLC4A1,Disease or Syndrome
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,1.047800e-06,0.30,C0001430,0.578,NaN,6948,C04,group,P20062,Neoplasms,Adenoma,NaN,2007.0,CURATED,1.000,0.615,NaN,2007.0,TCN2,Neoplastic Process
454,2.095700e-32,0.30,C0001430,0.419,Transporter,9429,C04,group,Q9UNQ0,Neoplasms,Adenoma,NaN,2011.0,CURATED,1.000,0.885,DTO_05007405,2011.0,ABCG2,Neoplastic Process
455,1.000000e+00,0.30,C0001546,0.552,Epigenetic regulator,10014,F03,group,Q9UQL6,Mental Disorders,Adjustment Disorders,NaN,2007.0,CURATED,1.000,0.769,DTO_05007378,2007.0,HDAC5,Mental or Behavioral Dysfunction
456,8.919400e-01,0.30,C0001618,0.636,Enzyme modulator,2771,C04;C19,group,P04899,Neoplasms; Endocrine System Diseases,Tumors of Adrenal Cortex,NaN,NaN,CURATED,NaN,0.577,DTO_05007584,NaN,GNAI2,Neoplastic Process


## Задание 1

In [ ]:
#Формируем таблицу генов по наименованию болезни
disease_db = dis.loc[dis['disease_name'] == 'Apert syndrome']
disease_db

,gene_pli,score,diseaseid,gene_dsi,protein_class_name,geneid,disease_class,disease_type,uniprotid,disease_class_name,disease_name,el,year_initial,source,ei,gene_dpi,protein_class,year_final,gene_symbol,disease_semantic_type
2,0.99731,0.80,C0001193,0.380,Kinase,2263,C16;C05,disease,P21802,"Congenital, Hereditary, and Neonatal Diseas...",Apert syndrome,NaN,1994.0,CURATED,0.976,0.808,DTO_03300101,2019.0,FGFR2,Congenital Abnormality
126,0.34491,0.31,C0001193,0.397,Transcription factor,7291,C16;C05,disease,Q15672,"Congenital, Hereditary, and Neonatal Diseas...",Apert syndrome,NaN,2005.0,CURATED,1.000,0.846,DTO_05007542,2013.0,TWIST1,Congenital Abnormality


In [ ]:
#Получаем список ID генов по наименованию болезни из таблицы
gene_id_list = disease_db['geneid'].tolist()
gene_id_list

[2263, 7291]

## Процедуры запроса необходимой информации из NCBI

In [ ]:
#Запрашиваем ID хромосомы из NCBI через параметр 'gi_id'.
#Эта процедура применяется в общей процедуре сбора информации по генам(см. ячейку ниже)
def search_chromosome(gi_id):
    handle = Entrez.esummary(db="nucleotide", id=gi_id)
    record = Entrez.read(handle)
    handle.close()
    return record[0]['AccessionVersion']

In [ ]:
def search_gene_name(gene_id):
  handle = Entrez.esummary(db="gene", id=gene_id)
  record = Entrez.read(handle)
  handle.close()
  return record['DocumentSummarySet']['DocumentSummary'][0]['Name']
  # print(record['DocumentSummarySet']['DocumentSummary'][0]['Name'])

In [ ]:
#Получаем из NCBI информацию о генах из списка 'gene_id_list', в том числе ID хромосомы.
def get_gene_info(gene_id_list):
  gene_info = []
  for gene_id in gene_id_list:
    handle = Entrez.efetch(db="gene", id=gene_id, rettype = 'xml')
    gene_locus = ET.parse(handle).getroot().find("Entrezgene/Entrezgene_locus")
    region = gene_locus.find("Gene-commentary/Gene-commentary_seqs/Seq-loc/Seq-loc_int/Seq-interval")
    start = int(region.find("Seq-interval_from").text) + 1
    end = int(region.find("Seq-interval_to").text) + 1
    gi_id = region.find("Seq-interval_id/Seq-id/Seq-id_gi").text
    strand = region.find("Seq-interval_strand/Na-strand").get("value")
    # print(gene_id, start, end, gi_id, strand)
    gene_info.append({'gene_name':search_gene_name(gene_id), 'gene_id':gene_id, 'interval_from':start, 'interval_to':end, 'gi_id':gi_id, 'nucleotide':search_chromosome(gi_id), 'strand':strand})
  return gene_info

In [ ]:
%%time
#Формируем список по всем генам
gene_list = get_gene_info(gene_id_list)
for i in gene_list:
  print(i)

{'gene_name': 'FGFR2', 'gene_id': 2263, 'interval_from': 121478330, 'interval_to': 121598458, 'gi_id': '568815588', 'nucleotide': 'NC_000010.11', 'strand': 'minus'}
{'gene_name': 'TWIST1', 'gene_id': 7291, 'interval_from': 19113047, 'interval_to': 19117636, 'gi_id': '568815591', 'nucleotide': 'NC_000007.14', 'strand': 'minus'}
CPU times: user 431 ms, sys: 47.2 ms, total: 478 ms
Wall time: 18.6 s


In [ ]:
#Представляем список генов в виде таблицы
df = pd.DataFrame(gene_list).head(1)
df['accession'] = df['nucleotide'].apply(lambda x: x.split('.')[0])
df['version'] = df['nucleotide'].apply(lambda x: x.split('.')[1])

In [ ]:
df.T

,0
gene_name,FGFR2
gene_id,2263
interval_from,121478330
interval_to,121598458
gi_id,568815588
nucleotide,NC_000010.11
strand,minus
accession,NC_000010
version,11


In [ ]:
#Получаем нуклеотидную последовательность гена. Параметры берем из 'gene_list'
def get_dna_sequence(gene_seq_keys_list):
  for nucleotid in gene_seq_keys_list:
    nucleotid_id = nucleotid['nucleotide']
    start = nucleotid['start']
    end = nucleotid['end']
    handle = Entrez.efetch(db="nucleotide",
                        id=nucleotid_id, 
                        rettype = 'fasta',
                        retmode = 'text')
    #загружаем нуклеотидную последовательность хромосомы
    chromosome = SeqIO.read(handle, 'fasta')
    gene_sym = chromosome.seq[start:end]
    print(f'Gene ID: {nucleotid["gene_id"]}')
    print('Chromosome description: ' + chromosome.description)
    print('Gene sequence: ' + gene_sym)
    print('Gene sequence length: ' + str(len(gene_sym)) + '\n')
    print(('=' * 100) + '\n')

In [ ]:
%%time
get_dna_sequence(gene_list)

Gene ID: 2263
Chromosome description: NC_000010.11 Homo sapiens chromosome 10, GRCh38.p14 Primary Assembly
Gene sequence: ...
Gene sequence length: 120128

Gene ID: 7291
Chromosome description: NC_000007.14 Homo sapiens chromosome 7, GRCh38.p14 Primary Assembly
Gene sequence: TTCTGCTCCTTAACATTATTTTATTAATAATACGTTCCACTTTGGGGAGGGGAACCTGTAACTGTGCGTGAATTATCATCTGGACATGTGCTTCTGTGAACAAAAGGGGAAATGTGCATATGGGTAAAAAAATTACTGCTTTCTTATAACTAAAGGCTATCAATCTGACCAACTTATAAATAAAAGTAATCAATAGCACAGTAAAGGAAACATTAAATTTTCTACAATTTACCCAAGACAATTTAATACAATACTTTCCCCTAGCAAATGCATTCTGAAGGAAAAAGATCCCAAGTGAACCAACACATTAATTGGAAAGCCTTAGCTTCTTATTTCCGAAAACAAGAGGTAATTCATTCTATGTGTTCTTTAAATAAGGGTAGGTAGCAAAAATAAAATGAAATAACATAAAATCAAAAGAAGATATAATGGTGGAGGAAAAGAAACAGGTGGCTTCATTTCTATTCTAAAAAAGAATTTCACGATTTATGTCTTGTGTATACAGGGGTAGGGATTAGAATCTGCAAGCTGGAAACTGCATTCCAGTAGTTAATATTTTTCATTGTGCAGTCGCCTCATTCAAAATAAAATTACCACCTTTTTCATTCATTTATTTCTCTGAAAATCTGGGGGGAAACATTACCTTCAGATTAGTATTTAATATTACTATAAATATGGCATTTGCTTTTCTCCAAAGTGTTTCCCCTTATGAAAAAAGTAGAA

### Получаем список генов по наименованию болезни через процедуру без использования файла `gene_disease_associations.tsv`

In [ ]:
def get_genes_by_disease(disease, retmax):
  disease_name = disease #Вносим название болезни во внутреннюю переменную
  handle = Entrez.esearch(db="gene", term=disease_name, retmax = retmax) #'retmax' Ограничиваем количество генов для вывода
  record = Entrez.read(handle)
  gene_ids = record["IdList"]
  print(gene_ids)
  print(len(gene_ids))
  return gene_ids

In [ ]:
gene_id_list = get_genes_by_disease('Apert syndrome', 30)
# print(len(set(gene_id_list)))
gene_list = get_gene_info(gene_id_list)
print()
for item in gene_list:
  print(item)
print ('\n')
get_dna_sequence(gene_list)

['2263', '21943', '14183', '14165', '23988', '16973', '16974', '2255']
8

{'gene_id': '2263', 'start': 121478330, 'end': 121598458, 'gi_id': '568815588', 'nucleotide': 'NC_000010.11', 'strand': 'minus'}
{'gene_id': '21943', 'start': 78514886, 'end': 78545483, 'gi_id': '1877089955', 'nucleotide': 'NC_000080.7', 'strand': 'minus'}
{'gene_id': '14183', 'start': 129764181, 'end': 129868538, 'gi_id': '1877089962', 'nucleotide': 'NC_000073.7', 'strand': 'minus'}
{'gene_id': '14165', 'start': 118851235, 'end': 118929109, 'gi_id': '1877089956', 'nucleotide': 'NC_000079.7', 'strand': 'plus'}
{'gene_id': '23988', 'start': 20563335, 'end': 20575008, 'gi_id': '1877089960', 'nucleotide': 'NC_000075.7', 'strand': 'plus'}
{'gene_id': '16973', 'start': 3634825, 'end': 3736574, 'gi_id': '1877089950', 'nucleotide': 'NC_000085.7', 'strand': 'minus'}
{'gene_id': '16974', 'start': 134423439, 'end': 134543876, 'gi_id': '1877089963', 'nucleotide': 'NC_000072.7', 'strand': 'minus'}
{'gene_id': '2255', 'start'

In [ ]:
disease_name = 'Breast cancer' #Вносим название болезни во внутреннюю переменную
handle = Entrez.esearch(db="gene", term=disease_name, retmax = 60) #'retmax' Ограничиваем количество генов для вывода
record = Entrez.read(handle)
# gene_ids = record["IdList"]
# print(gene_ids)
print(record)

{'Count': '26411', 'RetMax': '60', 'RetStart': '0', 'IdList': ['128887998', '128886574', '128886028', '128878594', '128874242', '128874134', '128867508', '128858730', '7157', '128894551', '128886603', '128878031', '128872539', '128864591', '128862217', '1956', '348', '7124', '7422', '3569', '7040', '4524', '22059', '2064', '3091', '2099', '3586', '351', '1636', '6774', '672', '3845', '627', '673', '4318', '21898', '1401', '367', '9370', '7421', '207', '5243', '4790', '29126', '3123', '3553', '1499', '5728', '1029', '21926', '7099', '6622', '5743', '4846', '1080', '1312', '7015', '3576', '4609', '4137'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'breast cancer[All Fields]', 'Field': 'All Fields', 'Count': '26411', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'breast cancer[All Fields]'}


### !!!Вопрос: gi_id - это ID чего?

__*Комментарии преподавателя:*__<br>
*Дарья Бородко・Преподаватель<br>*

Доброе утро! Все супер
На ваш вопрос о ```gi_id```:  
>*A GI number (for GenInfo Identifier, sometimes written in lower case, " gi ") is a simple series of digits that are assigned consecutively to each sequence record processed by NCBI. The GI number bears no resemblance to the Version number of the sequence record. Each time a sequence record is changed, it is assigned a new GI number.*  

Номер GI (для идентификатора GenInfo, иногда пишется строчными буквами «gi») представляет собой простую серию цифр, которые последовательно присваиваются каждой записи последовательности, обрабатываемой NCBI. Номер GI не имеет ничего общего с номером версии записи последовательности. При каждом изменении записи последовательности ей присваивается новый номер GI.
  
>*A nucleotide sequence GI number is shown in the VERSION field of the database record. A protein sequence GI number is shown in the VERSION field of a protein database record, and is cross-referenced in the CDS/db_xref field of a nucleotide database record.*

Номер GI нуклеотидной последовательности отображается в поле VERSION записи базы данных. Номер GI последовательности белка показан в поле VERSION записи базы данных белков, и на него есть перекрестная ссылка в поле CDS/db_xref записи базы данных нуклеотидов.